# Introduction 

Ce notebook présente une approche pour déterminer les couleurs d'une image initialement en noir et blanc, en utilisant des algorithmes de Machine Learning.

Ce projet s'intègre dans le cadre du cours I3 Python Scientifique de Centrale Méditerannée. Il a été mené par Thomas Roujou et Mathis Rezaï.

Il suit le plan suivant : 

I. Une première approche via réseau de neurones avec la bibliothèque tensorflow

1. Elaboration de réseau de neurones

2. Entrainement du modèle

3. Visualisation

II. Autre approche ?

Conclusion

# I. Une première approche via réseau de neurones avec la bibliothèque tensorflow

Tout d'abord nous devons importer toutes les librairies qui nous seront nécessaires pour cette partie

In [6]:
import sys
import numpy as np
from skimage import io, color
from skimage.transform import resize
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
image_path = 'images/mer.jpg'
image_color = io.imread(image_path)
image_color = resize(image_color, (256, 256), anti_aliasing=True)  # Redimensionne à 256x256
image_color = (image_color * 255).astype(np.uint8)  # Reconvertir en entier

FileNotFoundError: No such file: 'c:\Users\mathi\github\PY-Image-Colorization\notebook\images\mer.jpg'

In [7]:
image_path = 'images/mer.jpg'
image_color = io.imread(image_path)
image_color = resize(image_color, (256, 256), anti_aliasing=True)  # Redimensionne à 256x256
image_color = (image_color * 255).astype(np.uint8)  # Reconvertir en entier

FileNotFoundError: No such file: 'c:\Users\mathi\github\PY-Image-Colorization\notebook\images\mer.jpg'

Ensuite on utilise la fonction rgb2gray qui transforme l'image en couleurs en image en noir et blanc. Pour cela elle utilise la convention suivante (donner la formule exacte)

In [ ]:
image_gray = color.rgb2gray(image_color)